# Feed Forward




In [1]:
import torch
torch.set_default_tensor_type(torch.cuda.FloatTensor)
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import random
from tqdm import tqdm_notebook as tqdm


"""Change to the data folder"""
new_path = "./new_train/new_train/"

val_path = "./new_val_in/new_val_in/"

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        random.shuffle(self.pkl_list)
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data
    
    def capIt(self, a, b):
        self.pkl_list = self.pkl_list[a:b]


# intialize a dataset
train_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=new_path)
val_dataset  = ArgoverseDataset(data_path=val_path)

In [3]:
test_dataset.capIt(201000,202008)

In [4]:
train_dataset.capIt(0,200000)

In [5]:
print(len(train_dataset))
print(len(test_dataset))

200000
1008


In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

device = 'cuda'

class fcNetwork(nn.Module):

    def __init__(self, i_size, o_size, hidden , batchsize):
        super().__init__()
        
        self.fc_out = nn.Linear(hidden * 2, o_size * 2)
        self.fc_hidden = nn.Linear(i_size * 2, hidden * 2)
        
        
    def forward(self, x):
        
        out = self.fc_hidden(x)
        out = F.relu(self.fc_out(out))
        return out
        
    def init_hidden(self, batch_size, n_layers, hidden_dim):
        hidden = torch.zeros(n_layers, batch_size, hidden_dim).to(device)
        return hidden

In [7]:
batch_sz = 16
maxVal = 1500
def collateFC(batch):

    inp = []
    out = []
    index = []
    for scene in batch:
        inp = inp + [[scene['p_in'][i]] for i in range(60) if scene['track_id'][i, 0, 0] == scene['agent_id']]
        out = out + [[scene['p_out'][i]] for i in range(60) if scene['track_id'][i, 0, 0] == scene['agent_id']]
        index = index + [i for i in range(60) if scene['agent_id'] == scene['track_id'][i, 0, 0]]
    inp = torch.FloatTensor(inp).squeeze(1)
    out = torch.FloatTensor(out).squeeze(1)

    return[inp, out, index]



fc_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=collateFC, num_workers=0)
fc_test = DataLoader(test_dataset,batch_size=batch_sz, shuffle = False, collate_fn=collateFC, num_workers=0)

In [8]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10000):
    model.train()
    iterator = tqdm(fc_loader, total=int(len(fc_loader)))
    counter = 0
    for batch_idx, (inp, target, index) in enumerate(iterator):
        inp, target = inp.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        a = torch.zeros(1)
        inp = inp.reshape(batchSize, -1)
        target = target.reshape(batchSize, -1)
        
        inp = inp/maxVal
        target = target/maxVal        
        output = model(inp)

   
        output.to(device)
        target.to(device)
        loss = criterion(output * maxVal, target * maxVal)
        loss.backward()
        optimizer.step()
        counter += 1
        iterator.set_postfix(loss=(loss.item()*inp.size(0) / (counter * fc_loader.batch_size)))

In [9]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    i = 0
    with torch.no_grad():
        for inp, target, index in fc_test:
            inp, target = inp.to(device), target.to(device)
            
            inp = inp.reshape(batchSize, -1)
            target = target.reshape(batchSize, -1)
            inp = inp/maxVal
            target = target/maxVal
            a = torch.zeros(1)
            output = model(inp)

             #reshape output and target to check loss only on agent car

            test_loss += criterion(output * maxVal , target * maxVal)# sum up batch loss
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(fc_test.dataset),
        100. * correct / len(fc_test.dataset)))

In [10]:
lr = 0.001
n_epochs = 5
batchSize = 16

newModel = fcNetwork(19, 30, 24, 16)
newModel = newModel.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(newModel.parameters(), lr=lr, weight_decay = 1e-4)
criterion.to(device)

MSELoss()

In [11]:
# Training Run
for epoch in range(1, n_epochs + 1):
    train(newModel, device, fc_loader, optimizer, epoch)
    test(newModel, device, fc_test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until




Test set: Average loss: 28.6860, Accuracy: 0/1008 (0%)





Test set: Average loss: 9.4101, Accuracy: 0/1008 (0%)





Test set: Average loss: 6.0304, Accuracy: 0/1008 (0%)





Test set: Average loss: 6.1857, Accuracy: 0/1008 (0%)





Test set: Average loss: 5.1318, Accuracy: 0/1008 (0%)



In [12]:
batch_sz = 16


def collateVal(batch):

    inp = []
    index = []
    scene1 = []
    for scene in batch:
        inp = inp + [[scene['p_in'][i]] for i in range(60) if scene['track_id'][i, 0, 0] == scene['agent_id']]
        index = index + [i for i in range(60) if scene['agent_id'] == scene['track_id'][i, 0, 0]]
        scene1 = scene1 + [scene['scene_idx']]
    inp = torch.FloatTensor(inp).squeeze(1)

    return[inp, index, scene1]



val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=collateVal, num_workers=0)


In [13]:
## test file writing
import csv



header = ['ID','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18',
          'v19','v20','v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35'
          ,'v36','v37','v38','v39','v40','v41','v42','v43','v44','v45','v46','v47','v48','v49','v50','v51','v52',
          'v53','v54','v55','v56','v57','v58','v59','v60']

with open('./submission.csv', mode = 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(header)

    
    
    for inp, index, scene in val_loader:
        inp  = inp.to(device)
        
        inp = inp
        a = torch.zeros(1)
        inp = inp.reshape(batchSize, -1)
        output = newModel(inp)
 
         #reshape output and target to check loss only on agent car

        for i in range(batch_sz):
            writeLine = []
            writeLine.append(str(scene[i]))
            temp = output.reshape(batch_sz, -1)
            
            temp = temp[i]
            
            writeLine = writeLine + temp.tolist()
        
            if(len(writeLine) != 61):
                print("list error")
                
            writer.writerow(writeLine)
            #print(index)
        
        
print("Finished Writing")

Finished Writing


# MLP Regresser from SKLearn

In [6]:
from sklearn.neural_network import MLPRegressor

In [7]:
regr = MLPRegressor(random_state=1, max_iter=500)

In [38]:
batch_sz = 16
def collateFC(batch):

    inp = []
    out = []
    index = []
    for scene in batch:
        inp = inp + [[scene['p_in'][i]] for i in range(60) if scene['track_id'][i, 0, 0] == scene['agent_id']]
        out = out + [[scene['p_out'][i]] for i in range(60) if scene['track_id'][i, 0, 0] == scene['agent_id']]
        index = index + [i for i in range(60) if scene['agent_id'] == scene['track_id'][i, 0, 0]]
    inp = torch.FloatTensor(inp).squeeze(1)
    out = torch.FloatTensor(out).squeeze(1)

    return[inp, out, index]



train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=collateFC, num_workers=0)
test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = False, collate_fn=collateFC, num_workers=0)

In [22]:
from tqdm import tqdm_notebook as tqdm

x_train = []
x_labels = []
y_train = []
y_labels = []
iterator = tqdm(train_loader, total=int(len(train_loader)))
for i, [inp, out, index] in enumerate(iterator):
    for i in range(batch_sz):
        x_train.append(inp.reshape(batch_sz, -1)[i, ::2].tolist())
        x_labels.append(out.reshape(batch_sz, -1)[i, ::2].tolist())
        y_train.append(inp.reshape(batch_sz, -1)[i, 1::2].tolist())
        y_labels.append(out.reshape(batch_sz, -1)[i, 1::2].tolist())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [25]:
regr.fit(x_train, x_labels)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=500,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [41]:
from tqdm import tqdm_notebook as tqdm

x_test = []
y_test = []
x_test_labels = []
y_test_labels = []
iterator = tqdm(test_loader, total=int(len(test_loader)))
for i, [inp, out, index] in enumerate(iterator):
    for i in range(batch_sz):
        x_test.append(inp.reshape(batch_sz, -1)[i, ::2].tolist())
        x_test_labels.append(out.reshape(batch_sz, -1)[i, ::2].tolist())
        y_test.append(inp.reshape(batch_sz, -1)[i, 1::2].tolist())
        y_test_labels.append(out.reshape(batch_sz, -1)[i, 1::2].tolist())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [42]:
regr.score(x_test, x_test_labels)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.9999929410770512

In [90]:
batch_sz = 16
def collateTest(batch):

    inp = []
    index = []
    scene1 = []
    for scene in batch:
        inp = inp + [[scene['p_in'][i]] for i in range(60) if scene['track_id'][i, 0, 0] == scene['agent_id']]
        index = index + [i for i in range(60) if scene['agent_id'] == scene['track_id'][i, 0, 0]]
        scene1 = scene1 + [scene['scene_idx']]
    inp = torch.FloatTensor(inp).squeeze(1)

    return[inp, index, scene1]



val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=collateTest, num_workers=0)


In [91]:
from tqdm import tqdm_notebook as tqdm

x_val = []
y_val = []
sceneOrder = []
iterator = tqdm(val_loader, total=int(len(val_loader)))
for i, [inp, index, scene] in enumerate(iterator):
    for i in range(batch_sz):
        x_val.append(inp.reshape(batch_sz, -1)[i, ::2].tolist())
        y_val.append(inp.reshape(batch_sz, -1)[i, 1::2].tolist())
        sceneOrder.append(scene[i])


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [53]:
x_pred = regr.predict(x_val)

In [92]:
len(sceneOrder)

3200

In [55]:
regr2 = MLPRegressor(random_state=1, max_iter=500)
regr2.fit(y_train, y_labels)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=500,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [93]:
regr2.score(y_test, y_test_labels)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.9999623450878169

In [57]:
y_pred = regr2.predict(y_val)

In [97]:
sceneOrder

[11528,
 21811,
 20057,
 7040,
 12725,
 6297,
 32423,
 26325,
 22473,
 456,
 13382,
 34520,
 33215,
 20363,
 28855,
 39152,
 10019,
 12043,
 8491,
 19771,
 36718,
 38600,
 9707,
 34017,
 12828,
 984,
 35773,
 19686,
 28459,
 6478,
 18801,
 10909,
 31882,
 36890,
 7633,
 7784,
 35434,
 15551,
 25715,
 18368,
 19936,
 31818,
 25016,
 21735,
 20887,
 8879,
 21037,
 9836,
 24618,
 5853,
 22197,
 977,
 12739,
 33049,
 21209,
 16411,
 7178,
 20302,
 15391,
 24854,
 26415,
 9844,
 28296,
 4611,
 21132,
 3854,
 9262,
 36711,
 35823,
 24764,
 25134,
 38538,
 15459,
 31833,
 20845,
 17276,
 3242,
 23429,
 38717,
 25552,
 33548,
 16441,
 28484,
 9337,
 31969,
 3588,
 3633,
 32864,
 968,
 15276,
 26421,
 25566,
 18141,
 32404,
 12867,
 4556,
 31485,
 25028,
 23518,
 30416,
 23281,
 23168,
 9208,
 37751,
 4743,
 32125,
 17273,
 24308,
 2157,
 14357,
 34521,
 11227,
 34150,
 27240,
 13997,
 35657,
 12,
 27570,
 62,
 29594,
 3250,
 30526,
 14211,
 14511,
 16685,
 32619,
 6261,
 33792,
 14246,
 23670,

In [98]:
import pandas as pd
import numpy as np

dfc = pd.DataFrame(columns=np.arange(61))
dfc[0] = list(sceneOrder)
dfc

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,11528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,16905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3196,28089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3197,6857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3198,16152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
x_pred[0]

array([1881.73164028, 1879.92807411, 1878.23778084, 1878.29725071,
       1877.23252327, 1876.02236407, 1875.27236834, 1875.56675537,
       1873.22633314, 1871.74831349, 1871.42963322, 1870.18339108,
       1869.41651461, 1869.39937392, 1867.53540196, 1867.04908077,
       1865.79775967, 1866.35007988, 1864.56666823, 1863.15587558,
       1862.25597992, 1861.77892016, 1861.02735184, 1860.00247667,
       1858.00110568, 1858.26733218, 1856.21930268, 1856.55248921,
       1855.58248574, 1855.68861905])

In [100]:
y_pred[0]

array([486.03842717, 485.22602599, 484.91678638, 483.60740775,
       483.55512378, 482.33708484, 483.03563836, 482.54847685,
       481.15359872, 480.21359941, 479.45981897, 479.21355794,
       478.22688429, 477.82158985, 477.41905246, 477.69165354,
       476.43084328, 475.97100509, 475.71066906, 474.34527937,
       474.5781366 , 473.26888016, 472.66814252, 471.62088233,
       470.89379099, 470.84138041, 469.39396854, 470.44456683,
       469.98686338, 468.64036001])

In [101]:
for i in range(0,30):
    dfc[(i * 2) + 1] = x_pred[:, i]
    dfc[(2 * i) + 2] = y_pred[:, i]

In [102]:
dfc

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,11528,1881.731640,486.038427,1879.928074,485.226026,1878.237781,484.916786,1878.297251,483.607408,1877.232523,...,1858.267332,470.841380,1856.219303,469.393969,1856.552489,470.444567,1855.582486,469.986863,1855.688619,468.640360
1,21811,254.708490,1039.872984,254.585627,1038.889137,254.594566,1037.701746,254.783283,1036.928615,254.764632,...,255.513024,1018.542852,255.457484,1017.695371,255.634974,1016.850498,255.686676,1016.130810,255.772308,1014.982323
2,20057,2155.943395,709.099298,2154.467422,708.568434,2153.116022,707.799349,2153.609894,707.412619,2152.675959,...,2141.261714,697.260472,2139.560668,696.809420,2140.308635,696.350739,2139.635565,695.975627,2140.150727,695.287165
3,7040,201.537699,1535.678036,202.336697,1535.080775,202.712908,1534.049328,203.145452,1533.703812,204.116087,...,214.686196,1522.173753,214.943301,1521.623641,215.417004,1521.138921,215.592934,1520.721280,215.539759,1519.745928
4,12725,249.444091,1187.685341,249.306445,1186.087956,249.328818,1184.648237,249.536132,1183.043804,249.559171,...,250.708801,1155.998251,250.527023,1154.416109,250.811362,1153.720921,250.932460,1152.601370,250.994598,1150.809199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,16905,577.724882,1380.802520,577.452172,1380.104572,577.339700,1379.270221,577.610687,1378.841858,577.500031,...,577.284505,1368.610978,576.965026,1367.950790,577.278280,1367.584677,577.261276,1367.221699,577.468860,1366.473056
3196,28089,586.645465,1379.399500,586.368776,1380.394807,586.242682,1381.219545,586.563523,1382.421676,586.494351,...,587.277700,1408.285285,587.050579,1409.381164,587.417059,1410.319995,587.471244,1411.495963,587.665562,1412.260669
3197,6857,597.217246,1527.064490,596.686452,1527.016874,596.350725,1526.527276,596.472419,1526.807906,596.089304,...,592.180505,1528.450841,591.825208,1528.542128,591.854611,1528.538372,591.601316,1528.647933,591.734306,1528.216409
3198,16152,1745.845307,469.177597,1744.497498,468.534756,1742.896187,468.133884,1743.119558,467.242674,1742.227691,...,1726.336609,455.579601,1724.443274,454.601886,1725.104752,455.027369,1724.272582,454.714193,1724.361497,453.652381


In [103]:
header = ['ID','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18',
          'v19','v20','v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35'
          ,'v36','v37','v38','v39','v40','v41','v42','v43','v44','v45','v46','v47','v48','v49','v50','v51','v52',
          'v53','v54','v55','v56','v57','v58','v59','v60']
dfc.columns = (header )

In [104]:

dfc

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
0,11528,1881.731640,486.038427,1879.928074,485.226026,1878.237781,484.916786,1878.297251,483.607408,1877.232523,...,1858.267332,470.841380,1856.219303,469.393969,1856.552489,470.444567,1855.582486,469.986863,1855.688619,468.640360
1,21811,254.708490,1039.872984,254.585627,1038.889137,254.594566,1037.701746,254.783283,1036.928615,254.764632,...,255.513024,1018.542852,255.457484,1017.695371,255.634974,1016.850498,255.686676,1016.130810,255.772308,1014.982323
2,20057,2155.943395,709.099298,2154.467422,708.568434,2153.116022,707.799349,2153.609894,707.412619,2152.675959,...,2141.261714,697.260472,2139.560668,696.809420,2140.308635,696.350739,2139.635565,695.975627,2140.150727,695.287165
3,7040,201.537699,1535.678036,202.336697,1535.080775,202.712908,1534.049328,203.145452,1533.703812,204.116087,...,214.686196,1522.173753,214.943301,1521.623641,215.417004,1521.138921,215.592934,1520.721280,215.539759,1519.745928
4,12725,249.444091,1187.685341,249.306445,1186.087956,249.328818,1184.648237,249.536132,1183.043804,249.559171,...,250.708801,1155.998251,250.527023,1154.416109,250.811362,1153.720921,250.932460,1152.601370,250.994598,1150.809199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,16905,577.724882,1380.802520,577.452172,1380.104572,577.339700,1379.270221,577.610687,1378.841858,577.500031,...,577.284505,1368.610978,576.965026,1367.950790,577.278280,1367.584677,577.261276,1367.221699,577.468860,1366.473056
3196,28089,586.645465,1379.399500,586.368776,1380.394807,586.242682,1381.219545,586.563523,1382.421676,586.494351,...,587.277700,1408.285285,587.050579,1409.381164,587.417059,1410.319995,587.471244,1411.495963,587.665562,1412.260669
3197,6857,597.217246,1527.064490,596.686452,1527.016874,596.350725,1526.527276,596.472419,1526.807906,596.089304,...,592.180505,1528.450841,591.825208,1528.542128,591.854611,1528.538372,591.601316,1528.647933,591.734306,1528.216409
3198,16152,1745.845307,469.177597,1744.497498,468.534756,1742.896187,468.133884,1743.119558,467.242674,1742.227691,...,1726.336609,455.579601,1724.443274,454.601886,1725.104752,455.027369,1724.272582,454.714193,1724.361497,453.652381


In [105]:
dfc.to_csv('sub_dfc.csv', index=False)

In [116]:
m = nn.MSELoss()
m(torch.tensor(regr.predict(y_test), dtype = torch.double), torch.tensor(y_test_labels, dtype = torch.double))

tensor(21.6318, dtype=torch.float64)